* We have all the tools we need to write programs: variables & lists, loops, conditionals, functions
* But we don't know how to tell if we're getting the right answer, and how to ensure that we keep getting the right answer even as we change the program
* To do that, we insert tests into the program so it can check its own operation as  it runs
* As in real carpentry, when the time saved by measuring lumber carefully before cutting far outweighs the time it takes to make the measurements, the time saved by adding these checks is much greater than the time it takes to add them

# Assertions
* Assertions check that a statement is always true -- if yes, do nothing; if no, halt with error

In [1]:
numbers = [1.5, 2.3, 0.7, -0.001, 4.4]
total = 0.0
for n in numbers:
    total += n
print('total is:', total)

AssertionError: Data should only contain positive values

Three kinds of assertions:
* Precondition: must be true at start of function for it to work properly
* Postcondition: something that function guarantees is true when it finishes
* Invariant: something always true at a point inside a function

In [ ]:
numbers = [1.5, 2.3, 0.7, -0.001, 4.4]

assert len(numbers) > 0, 'No numbers given'
total = 0.0
for n in numbers:
    assert n > 0, 'Total is not increasing'
    total += n
assert max(numbers) < total, 'Total is bigger than any one number'
print('total is', total)

In [ ]:
def normalize_rectangle(rect):
    '''Normalizes a rectangle so that it is at the origin and 1.0 units long on its longest axis.'''
    assert len(rect) == 4, 'Rectangles must contain 4 coordinates'
    x0 = rect[0]
    y0 = rect[1]
    x1 = rect[2]
    y1 = rect[3]
    assert x0 < x1, 'Invalid X coordinates'
    assert y0 < y1, 'Invalid Y coordinates'

    dx = x1 - x0
    dy = y1 - y0
    if dx > dy:
        scaled = dx / dy
        upper_x, upper_y = 1.0, scaled
    else:
        scaled = dx / dy
        upper_x, upper_y = scaled, 1.0

    assert 0 < upper_x <= 1.0, 'Calculated upper X coordinate invalid'
    assert 0 < upper_y <= 1.0, 'Calculated upper Y coordinate invalid'

    return (0, 0, upper_x, upper_y)

In [ ]:
print(normalize_rectangle( (0.0, 1.0, 2.0) )) # missing the fourth coordinate
print(normalize_rectangle( (4.0, 2.0, 1.0, 5.0) )) # X axis inverted

In [ ]:
# Okay -- tall and skinny
print(normalize_rectangle( (0.0, 0.0, 1.0, 5.0) ))

# Bad -- short and fat
print(normalize_rectangle( (0.0, 0.0, 5.0, 1.0) ))
# Line 10 should be float(dy) / dx

# Show my breakpoint consensus at this point.

# Why use assertions?

* I'm going to give you a grand unified theory of programming
* Computers are very fast but very stupid
* Everything we do can be reduced to operations like "add the value at a certain location in memory to the value at another location. If the result is more than 15, jump to the instructions located here in memory and start executing them. Otherwise, jump to this other set of instructions and run them."
    * So, if I send you a graph via e-mail, everything -- the generation of the graph, the sending, and so forth -- is ultimately reduced to those instructions
* There's no notion of functions, of objects, even of different data types -- there's no way, for example, of representing integers above a certain upper limit, or non-integer values that have a decimal component.
* That's the very stupid part -- but what about very fast? well, if you buy a CPU that operates at 2 GHz, it can execute 2 billion of those very simple instructions every second
* Programming is the art of describing the solution to a problem in such excruciating detail that even a machine as dumb as a computer can understand it
    * This is extremely apparent when you're programming at the extremely low level I described above. But in Python, we're doing the same thing, but allowing the programming language 
* Errors arise when we make assumptions about how we're describing the problem -- we're not good at thinking about problems at such a low level
* Assertions let us insert our assumptions into the code and make sure that they are, actually, true

# Exercise 1

Look at the function I'm writing. Tell me what each of the asserts does.

#1: at least one value given
#2: first value is non-negative
#3: total never decreases

In [9]:
def running(values):
    assert len(values) > 0 # Precondition
    result = [values[0]]
    assert result[0] >= 0 # Invariant
    for v in values[1:]:
        result.append(result[-1] + v)
        assert result[-1] >= result[-2] # Invariant
    assert len(result) == len(values) # Postcondition
    return result

running([1, 2, 3, 4])

[1, 3, 6, 10]

# Writing tests using assertions

We want to determine the overlap between different ranges -- i.e., their intersection.

![ranges](ranges.svg)

Here the overlap is [0, 2].
* We could try writing this function right now, then write a bunch of assertions afterward to test that it produces the correct output
* But we can also try flipping the problem around, so we write the tests first

In [ ]:
assert range_overlap([ (2.0, 3.0), (2.0, 4.0) ]) == (2.0, 3.0)
assert range_overlap([ (0.0, 1.0), (0.0, 2.0), (-1.0, 1.0) ]) == (0.0, 1.0)
assert range_overlap([ (0.0, 1.0) ]) == (0.0, 1.0)

In [ ]:
# No overlap
assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == None

# Overlap at single point -- should this be [1, 1] or None?
assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == None

In [2]:
# Now we can write our function
def range_overlap(ranges):
    '''Return common overlap among a set of [low, high] ranges.'''
    lowest = 0.0
    highest = 1.0
    for (low, high) in ranges:
        lowest = max(lowest, low)
        highest = min(highest, high)
    return (lowest, highest)

In [3]:
# Let's combine into single function
def test_range_overlap():
    assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == None
    assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == None
    assert range_overlap([ (0.0, 1.0) ]) == (0.0, 1.0)
    assert range_overlap([ (2.0, 3.0), (2.0, 4.0) ]) == (2.0, 3.0)
    assert range_overlap([ (0.0, 1.0), (0.0, 2.0), (-1.0, 1.0) ]) == (0.0, 1.0)
    
# Now run it
test_range_overlap()

AssertionError: 

# Exercise 2

Fix range_overlap. Rerun test_overlap after each change

In [ ]:
import numpy

def range_overlap(ranges):
    '''Return common overlap among a set of [low, high] ranges.'''
    if len(ranges) == 1: # only one entry, so return it
        return ranges[0]
    lowest = -numpy.inf # lowest possible number
    highest = numpy.inf # highest possible number
    for (low, high) in ranges:
        lowest = max(lowest, low)
        highest = min(highest, high)
    if lowest >= highest: # no overlap
        return None
    else:
        return (lowest, highest)